In [2]:
# needs to be run in an R kernel corresponds to the ClusteringCellsChen.R file
#  See here:  https://irkernel.github.io/
# install guide:  https://irkernel.github.io/installation/#binary-panel

#set propoer working dirrectory in pavlab server
setwd('/home/acampbell/PavLabEngrams/EngramCellClassifier/Chen2020_GSE152632')

#libraries
library(Seurat)
library(dplyr)
library(tidyverse)
library(ggplot2)

Attaching SeuratObject


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v stringr 1.4.0
v tidyr   1.2.0     v forcats 0.5.1
v readr   2.1.2     

-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [4]:
#Load the data and Metadata
chen2020_counts <- read.csv('GSE152632_GEO_mberkchen_TRAP2_counts.csv.gz', header = TRUE)
rownames(chen2020_counts) <- chen2020_counts$X
chen2020_counts <- chen2020_counts[,2:3531]
chen2020_meta <- read.csv( 'SraRunTable.txt', header = TRUE)

In [5]:
#Load the data and Metadata
chen2020_counts <- read.csv('GSE152632_GEO_mberkchen_TRAP2_counts.csv.gz', header = TRUE)
rownames(chen2020_counts) <- chen2020_counts$X
chen2020_counts <- chen2020_counts[,2:3531]
chen2020_meta <- read.csv( 'SraRunTable.txt', header = TRUE)

#add engram label, the label is burried in the cell count column names
chen2020_meta$engram_label <-  as.factor(sapply(as.character(colnames(chen2020_counts)), function(y) if (grepl("_pos_", y, fixed=TRUE)) "tdT+" else "tdT-"))

#create the condition label
condition_label <- chen2020_meta$source_name %>%
  sapply( function(y) if (grepl("Homecage", y, fixed=TRUE)) "Homecage")

condition_label <- chen2020_meta$source_name
condition_label[str_detect(condition_label, "Homecage")] = "Homecage"
condition_label[str_detect(condition_label, "Context-Only")] = "Context-Only"
condition_label[str_detect(condition_label, "Fear-Only")] = "Fear-Only"
condition_label[str_detect(condition_label, "Fear-Recall")] = "Fear-Recall"
chen2020_meta$condition_label <- condition_label

From the Chen et al., (2020) text:<br>
*"We applied standard algorithms for cell filtration, feature selection and dimensionality reduction. Briefly, genes that appeared in fewer than five cells, samples with fewer than 100 genes and samples with less than 50,000 reads were excluded from the analysis. Out of these cells, those with more than 30% of reads as ERCC, and more than 10% mitochondrial or 10% ribosomal were also excluded from analysis. Counts were log-normalized and then scaled where appropriate."*

In [8]:
table(chen2020_meta$condition_label)


Context-Only    Fear-Only  Fear-Recall     Homecage 
         912          841         1342          435 

In [ ]:
underexpressedgenes <- c()

for(i in c(1:dim(chen2020_counts)[1]) ){
    
    if (sum(chen2020_counts[i,]>0)>5){
    underexpressedgenes <- c(underexpressedgenes,i)
    }

}
underexpressedgenes

In [ ]:
underexpressedgenes <- c()

In [10]:
#adding cell bacrcodes from count data to rows of metadata for seurat
chen2020_counts <- chen2020_counts[!c(1:23355) %in% underexpressedgenes,]
rownames(chen2020_meta) <- colnames(chen2020_counts)

ERROR: Error in c(1:23355) %in% underexpressedgenes: object 'underexpressedgenes' not found
